input -> NN.module.forward() -> output, 网络内: 卷积 -> 非线性 -> 卷积 -> 非线性 -> 输出

PyTorch的nn.Module是所有神经网络模块的基类。你可以通过继承这个类来创建自己的神经网络模型：

- 定义模型：通过继承nn.Module类并在__init__方法中定义网络层
- 前向传播：在forward方法中定义数据的前向传播过程
- 模块嵌套：nn.Module允许模块嵌套，可以将子模块作为属性添加到父模块中，从而构建复杂的网络结构
- 参数管理：所有定义在nn.Module中的参数都会自动注册，并且可以通过parameters()方法访问
- 训练和评估模式：通过model.train()和model.eval()方法切换模型的训练和评估模式

In [ ]:
# NN模板
import torch
from torch import nn

class MyNN(nn.Module):  # 必须继承自nn.Module
    def __init__(self):
        super(MyNN,self).__init__()
        
    def forward(self,input):
        output = input + 1
        return output

mynn = MyNN()
x = torch.tensor(1.0)
output = mynn(x)
print(output)

PyTorch的卷积层（Convolution Layers）是构建卷积神经网络（CNN）的基础：

- Conv2d: 这是最常用的2D卷积层，用于处理图像数据。它的主要参数包括：
    - in_channels: 输入特征的通道数，例如灰度图像有1个通道，彩色图像有3个通道
    - out_channels: 卷积核的数量，即输出特征的通道数。
    - kernel_size: 卷积核的大小，可以是单个整数或一个整数元组。
    - stride: 卷积操作的步幅，控制卷积核移动的步长。
    - padding: 输入图像的填充方式，可以是整数或字符串（如’valid’或’same’）
    - dilation: 卷积核元素之间的间距
- Conv3d: 用于3D数据的卷积层，适用于视频或3D图像处理
- ConvTranspose2d: 反卷积层（或转置卷积层），常用于生成对抗网络（GAN）中的上采样操作。
- Groups: 控制输入和输出之间的连接方式。例如，groups=1表示所有输入通道与所有输出通道相连，而groups=in_channels表示每个输入通道与其自己的卷积核相连

tips: 卷积核可以理解为ouput之于input的投影(详见PyTorch官方文档-Convolution arithmetic)

这些卷积层可以通过torch.nn模块轻松定义和使用

In [ ]:
import torchvision
dataset = torchvision.datasets.CIFAR10(root=r"..\data\test",train=False,transform=torchvision.transforms.ToTensor())

In [ ]:
from torch.utils.data import DataLoader
daraloader = DataLoader(dataset=dataset,batch_size=64,drop_last=False)

In [ ]:
from torch.nn import Conv2d

class MyCNN(nn.Module):

    def __init__(self):
        super(MyCNN,self).__init__()
        self.conv1 = Conv2d(in_channels=3,out_channels=6,kernel_size=(3,3),stride=1,padding=0)

    def forward(self,x):
        x = self.conv1(x)
        return x
    
mycnn = MyCNN()
print(mycnn)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("logs")
step = 0
for data in daraloader:
    imgs,targets = data
    output = mycnn(imgs)
    # torch.Size([64,3,30,30])
    writer.add_images(tag="input",img_tensor=imgs,global_step=step)
    # torch.Size([64,6,30,30]),Tensorboard不好显示
    output = torch.reshape(input=output,shape=(-1,3,30,30))  # -> torch.Size([xxx,3,30,30])
    writer.add_images(tag="output",img_tensor=output,global_step=step)
    step += 1
writer.close()